In [25]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import optim
import torchvision

In [26]:
size = 512

train_db = torchvision.datasets.MNIST(
    root="data/mnist_data",
    train=True,
    download=True,
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.5,), (0.5,)),
        ]
    ),
)

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting data/mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to data/mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting data/mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to data/mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%


Extracting data/mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/mnist_data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100.0%

Extracting data/mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/mnist_data/MNIST/raw



In [ ]:
import torch.utils.data.dataloader


train_loader=torch.utils.data.DataLoader(train_db,batch_size=size,shuffle=True)